In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/low-resolution.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from torchvision import models
from torch import nn
import torch.optim as optim

from sklearn.metrics import accuracy_score

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
my_transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    # transforms.Grayscale(),
    transforms.Normalize([0.485], [0.229])])

dataset = datasets.CIFAR10("./data", True, my_transform, download=True)

labels = dataset.targets

# split the dataset into train/validation/test sets with stratification
train_idx, test_idx, train_labels, test_labels = train_test_split(
    range(len(dataset)), labels, test_size=0.1, stratify=labels, random_state=0)


# create the subsets from the dataset
train_data = torch.utils.data.Subset(dataset, train_idx)
test_data = torch.utils.data.Subset(dataset, test_idx)

# create the data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)

Files already downloaded and verified


### List of ImageNet and Stanford Dog Dataset Breeds to See if it Identifies a Dog

In [6]:
dog_breeds = ["affenpinscher",
    "afghan hound",
    "african hunting dog",
    "airedale",
    "american staffordshire terrier",
    "appenzeller",
    "australian terrier",
    "basenji",
    "basset",
    "beagle",
    "bedlington terrier",
    "bernese mountain dog",
    "black-and-tan coonhound",
    "blenheim spaniel",
    "bloodhound",
    "bluetick",
    "border collie",
    "border terrier",
    "borzoi",
    "boston bull",
    "bouvier des flandres",
    "boxer",
    "brabancon griffon",
    "briard",
    "brittany spaniel",
    "bull mastiff",
    "cairn",
    "cardigan",
    "chesapeake bay retriever",
    "chihuahua",
    "chow",
    "clumber",
    "cocker spaniel",
    "collie",
    "curly-coated retriever",
    "dandie dinmont",
    "dhole",
    "dingo",
    "doberman",
    "english foxhound",
    "english setter",
    "english springer",
    "entlebucher",
    "eskimo dog",
    "flat-coated retriever",
    "french bulldog",
    "german shepherd",
    "german short-haired pointer",
    "giant schnauzer",
    "golden retriever",
    "gordon setter",
    "great dane",
    "great pyrenees",
    "greater swiss mountain dog",
    "groenendael",
    "ibizan hound",
    "irish setter",
    "irish terrier",
    "irish water spaniel",
    "irish wolfhound",
    "italian greyhound",
    "japanese spaniel",
    "keeshond",
    "kelpie",
    "kerry blue terrier",
    "komondor",
    "kuvasz",
    "labrador retriever",
    "lakeland terrier",
    "leonberg",
    "lhasa",
    "malamute",
    "malinois",
    "maltese dog",
    "mexican hairless",
    "miniature pinscher",
    "miniature poodle",
    "miniature schnauzer",
    "newfoundland",
    "norfolk terrier",
    "norwegian elkhound",
    "norwich terrier",
    "old english sheepdog",
    "otterhound",
    "papillon",
    "pekinese",
    "pembroke",
    "pomeranian",
    "pug",
    "redbone",
    "rhodesian ridgeback",
    "rottweiler",
    "saint bernard",
    "saluki",
    "samoyed",
    "schipperke",
    "scotch terrier",
    "scottish deerhound",
    "sealyham terrier",
    "shetland sheepdog",
    "shih-tzu",
    "siberian husky",
    "silky terrier",
    "soft-coated wheaten terrier",
    "staffordshire bullterrier",
    "standard poodle",
    "standard schnauzer",
    "sussex spaniel",
    "tibetan mastiff",
    "tibetan terrier",
    "toy poodle",
    "toy terrier",
    "vizsla",
    "walker hound",
    "weimaraner",
    "welsh springer spaniel",
    "west highland white terrier",
    "whippet",
    "wire-haired fox terrier",
    "yorkshire terrier"]

### Test If Images Are Dogs

In [22]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
resnet = resnet.to(device)
resnet.eval()

weights = models.ResNet50_Weights.DEFAULT

y_predictions = []
y_true = []
test_len = len(test_loader)

for i, data in enumerate(test_loader):
    inputs, labels = data
    
    inputs = inputs.to(device)
    labels = labels.to(device)

    log_probabilities = resnet(inputs).squeeze(0)
    y_prediction = log_probabilities.argmax()
    y_predictions.append(y_prediction.cpu())
    prediction_name = weights.meta["categories"][y_prediction]

    # If image of dog and predicts a dog or if image is not a dog and predicts not a dog
    if ((labels[0] == 5) and (prediction_name.lower() in dog_breeds)) or ((labels[0] != 5) and (prediction_name.lower() not in dog_breeds)):
        y_true.append(y_prediction.cpu())
    else:
        y_true.append(torch.tensor(-1).cpu())


    if i % 100 == 0:
        print(f"{i} / {test_len}")

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0 / 5000
100 / 5000
200 / 5000
300 / 5000
400 / 5000
500 / 5000
600 / 5000
700 / 5000
800 / 5000
900 / 5000
1000 / 5000
1100 / 5000
1200 / 5000
1300 / 5000
1400 / 5000
1500 / 5000
1600 / 5000
1700 / 5000
1800 / 5000
1900 / 5000
2000 / 5000
2100 / 5000
2200 / 5000
2300 / 5000
2400 / 5000
2500 / 5000
2600 / 5000
2700 / 5000
2800 / 5000
2900 / 5000
3000 / 5000
3100 / 5000
3200 / 5000
3300 / 5000
3400 / 5000
3500 / 5000
3600 / 5000
3700 / 5000
3800 / 5000
3900 / 5000
4000 / 5000
4100 / 5000
4200 / 5000
4300 / 5000
4400 / 5000
4500 / 5000
4600 / 5000
4700 / 5000
4800 / 5000
4900 / 5000


In [23]:
accuracy_score(y_true, y_predictions)

0.9064